In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import csv
import os


In [42]:
#importing the dataset 
df=pd.read_csv('data.tsv', delimiter=',')
df.head()

,DB00357,P05108
0,DB02721,P00325
1,DB00773,P23219
2,DB07138,Q16539
3,DB08136,P24941
4,DB01242,P23975


In [44]:
df.to_csv('F-Dataset.csv',index=False,header=None)
g=nx.read_edgelist('F-Dataset.csv',delimiter=',',create_using=nx.DiGraph())
print(nx.info(g))


Name: 
Type: DiGraph
Number of nodes: 7341
Number of edges: 15137
Average in degree:   2.0620
Average out degree:   2.0620


In [45]:
#generate the 15138  missing edges and storing them
import random
myedges = set([])
r = csv.reader(open('F-Dataset.csv','r'))
edges = dict()
for edge in r:
    edges[(edge[0], edge[1])] = 1
while (len(myedges)<15138):
    a=random.randint(1,7341)
    b=random.randint(1,7341)
    tmp = edges.get((a,b),-1)
    if tmp == -1 and a!=b:
        try:
            if nx.shortest_path_length(g,source=a,target=b) >2: 
                myedges.add((a,b))
            else:
                continue  
        except:  
               myedges.add((a,b))              
    else:
        continue
len(myedges)

15138

In [47]:
from sklearn.model_selection import train_test_split

In [46]:
if (not os.path.isfile('train_data.csv')) and (not os.path.isfile('test_data.csv')):
    
    df_pos = pd.read_csv('F-Dataset.csv')
    df_neg = pd.DataFrame(list(myedges), columns=['drug', 'protein'])
    
    print("Number of nodes in the graph with edges", df_pos.shape[0])
    print("Number of nodes in the graph without edges", df_neg.shape[0])


In [6]:
#using the train_test_split to split our data to test and train
X_train_pos, X_test_pos, y_train_pos, y_test_pos  = train_test_split(df_pos,np.ones(len(df_pos)),test_size=0.2, random_state=9)
X_train_neg, X_test_neg, y_train_neg, y_test_neg  = train_test_split(df_neg,np.zeros(len(df_neg)),test_size=0.2, random_state=9)
#removing header and saving
X_train_pos.to_csv('train_data.csv',header=False, index=False)
X_test_pos.to_csv('test_data.csv',header=False, index=False)
X_train_neg.to_csv('train_neg.csv',header=False, index=False)
X_test_neg.to_csv('test_neg.csv',header=False, index=False)


#train data graph info
print("Number of nodes in the train data graph with edges", X_train_pos.shape[0],"=",y_train_pos.shape[0])
print("Number of nodes in the train data graph without edges", X_train_neg.shape[0],"=", y_train_neg.shape[0])
#test data graph info
print("Number of nodes in the test data graph with edges", X_test_pos.shape[0],"=",y_test_pos.shape[0])
print("Number of nodes in the test data graph without edges", X_test_neg.shape[0],"=",y_test_neg.shape[0])


Number of nodes in the train data graph with edges 12110 = 12110
Number of nodes in the train data graph without edges 12110 = 12110
Number of nodes in the test data graph with edges 3028 = 3028
Number of nodes in the test data graph without edges 3028 = 3028


In [102]:
del myedges

In [8]:
if (os.path.isfile('train_data.csv')) and (os.path.isfile('test_data.csv')):        
    train_graph=nx.read_edgelist('train_data.csv',delimiter=',',create_using=nx.DiGraph())
    test_graph=nx.read_edgelist('test_data.csv',delimiter=',',create_using=nx.DiGraph())
    print(nx.info(train_graph))
    print(nx.info(test_graph))

    # finding the unique nodes in the both train and test graphs
    train_nodes_pos = set(train_graph.nodes())
    test_nodes_pos = set(test_graph.nodes())

    trY_teY = len(train_nodes_pos.intersection(test_nodes_pos))
    trY_teN = len(train_nodes_pos - test_nodes_pos)
    teY_trN = len(test_nodes_pos - train_nodes_pos)
    print('---------------------------------------- ')
    

    print('nbr of nodes that are both in train and test: ',trY_teY)
    print('nbr of nodes only in train data: ',trY_teN)
    print('nbr of nodes only in test data: ',teY_trN)


Name: 
Type: DiGraph
Number of nodes: 6552
Number of edges: 12109
Average in degree:   1.8481
Average out degree:   1.8481
Name: 
Type: DiGraph
Number of nodes: 2775
Number of edges: 3028
Average in degree:   1.0912
Average out degree:   1.0912
---------------------------------------- 
nbr of nodes that are both in train and test:  1986
nbr of nodes only in train data:  4566
nbr of nodes only in test data:  789


In [10]:
if (not os.path.isfile('train_final.csv')) and \
(not os.path.isfile('test_final.csv')) and \
(not os.path.isfile('train_y.csv')) and \
(not os.path.isfile('test_y.csv')) and \
(os.path.isfile('train_data.csv')) and \
(os.path.isfile('test_data.csv')) and \
(os.path.isfile('train_neg.csv')) and \
(os.path.isfile('test_neg.csv')):
    
    X_train_pos = pd.read_csv('train_data.csv', names=['drug', 'gene'])
    X_test_pos = pd.read_csv('test_data.csv', names=['drug', 'gene'])
    X_train_neg = pd.read_csv('train_neg.csv', names=['drug', 'gene'])
    X_test_neg = pd.read_csv('test_neg.csv', names=['drug', 'gene'])

    X_train = X_train_pos.append(X_train_neg,ignore_index=True)
    y_train = np.concatenate((y_train_pos,y_train_neg))
    X_test = X_test_pos.append(X_test_neg,ignore_index=True)
    y_test = np.concatenate((y_test_pos,y_test_neg)) 
    
    X_train.to_csv('train_final.csv',header=False,index=False)
    X_test.to_csv('test_final.csv',header=False,index=False)
    pd.DataFrame(y_train.astype(int)).to_csv('train_y.csv',header=False,index=False)
    pd.DataFrame(y_test.astype(int)).to_csv('test_y.csv',header=False,index=False)

In [11]:
print("Data points in train data",X_train.shape)
print("Data points in test data",X_test.shape)
print("Shape of traget variable in train",y_train.shape)
print("Shape of traget variable in test", y_test.shape)

Data points in train data (24220, 2)
Data points in test data (6056, 2)
Shape of traget variable in train (24220,)
Shape of traget variable in test (6056,)
